# Thread Experiment

In [ ]:
import threading
import time

x = 5

def print_x():
    while True:
        print("x =", x)
        time.sleep(1)  # sleep for 1 sec

server = threading.Thread(target=print_x)
server.run()
time.sleep(2)
x = 10
time.sleep(2)
# should have printed x = 10 before exit
server.exit()

Error: Error: Jupyter cannot be started. Error attempting to locate jupyter: Error: Module 'notebook' not installed.